In [2]:
!pip install mysql-connector-python

In [3]:
import mysql.connector as connection
import pandas as pd
import numpy as np
mydb = connection.connect(host='127.0.0.1', database='coursera_assignment_schema', user='root',\
                          passwd='TuanIlyas',use_pure=True)

In [4]:
curr = mydb.cursor()

### Problem 1

##### Find the total number of crimes recorded in the CRIME table.

In [5]:
curr.execute('select count(*) from CHICAGO_CRIME_DATA')
crime_number = curr.fetchall()
print('The total number of crime recorded is {}'.format(*np.unique(crime_number)))

The total number of crime recorded is 353


### Problem 2

##### List community areas with per capita income less than 11000.

In [6]:
curr.execute('select COMMUNITY_AREA_NAME, PER_CAPITA_INCOME from CENSUS_DATA where PER_CAPITA_INCOME<11000')
comm_areas = curr.fetchall()
comm_areas_pd = pd.DataFrame()
for row in range(len(comm_areas)) :
    comm_area = comm_areas[row][0]
    income = comm_areas[row][1]
    comm_areas_pd = comm_areas_pd.append({'Community Area':comm_area,'Per Capita Income':income},ignore_index=True)
comm_areas_pd

,Community Area,Per Capita Income
0,West Garfield Park,10934.0
1,South Lawndale,10402.0
2,Fuller Park,10432.0
3,Riverdale,8201.0


### Problem 4 

##### List all kidnapping crimes involving a child

In [7]:
curr.execute('select CASE_NUMBER,DATE,BLOCK,DESCRIPTION from chicago_crime_data where PRIMARY_TYPE=\'KIDNAPPING\'')
kidnapping = curr.fetchall()
kidnapping_pd = pd.DataFrame()
for row in range(len(kidnapping)) :
    case_num = kidnapping[row][0]
    date = kidnapping[row][1]
    block = kidnapping[row][2]
    desc = kidnapping[row][3]
    kidnapping_pd = kidnapping_pd.append({'CASE_NUMBER':case_num,'DATE':date,'BLOCK':block,'DESCRIPTION':desc},ignore_index=True)
kidnapping_pd

,BLOCK,CASE_NUMBER,DATE,DESCRIPTION
0,050XX W VAN BUREN ST,HN144152,2007-01-26,CHILD ABDUCTION/STRANGER


### Problem 5
##### What kind of crimes were recorded at schools?

In [8]:
curr.execute("""select distinct(PRIMARY_TYPE) from chicago_crime_data 
             where LOCATION_DESCRIPTION like \'SCHOOL%\'""")
school_crime = curr.fetchall()
school_crime_pd = pd.DataFrame()
for row in range(len(school_crime)) :
    PRIM = school_crime[row][0]
    school_crime_pd = school_crime_pd.append({'PRIMARY_TYPE':PRIM},ignore_index=True)
school_crime_pd

,PRIMARY_TYPE
0,CRIMINAL DAMAGE
1,NARCOTICS
2,CRIMINAL TRESPASS


### Problem 6
##### List the average safety score for all types of schools.

In [9]:
curr.execute("""select `Elementary, Middle, or High School`, avg(SAFETY_SCORE) from chicago_public_schools 
             group by `Elementary, Middle, or High School`""")
safe_score = curr.fetchall()
safe_score_pd = pd.DataFrame()
for row in range(len(safe_score)) :
    school = safe_score[row][0]
    avg = safe_score[row][1]
    safe_score_pd = safe_score_pd.append({'SCHOOL_TYPE':school,'AVERAGE_SAFETY_SCORE':avg},ignore_index=True)
safe_score_pd

,AVERAGE_SAFETY_SCORE,SCHOOL_TYPE
0,49.4371,ES
1,56.8286,HS
2,53.0000,MS


### Problem 7
##### List 5 community areas with highest % of households below poverty line.

In [10]:
curr.execute("""SELECT COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY FROM census_data 
order by PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC LIMIT 5""")
poverty_comm = curr.fetchall()
poverty_comm_pd = pd.DataFrame()
for row in range(len(poverty_comm)) :
    comm_area = poverty_comm[row][0]
    percent = poverty_comm[row][1]
    poverty_comm_pd = poverty_comm_pd.append({'COMMUNITY_AREA_NAME':comm_area,'PERCENT_HOUSEHOLDS_BELOW_POVERTY':percent},ignore_index=True)
poverty_comm_pd

,COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
0,Riverdale,56.5
1,Fuller Park,51.2
2,Englewood,46.6
3,North Lawndale,43.1
4,East Garfield Park,42.4


### Problem 8 
###### Which community area(number) is most crime prone?

In [11]:
#QUERY = """select COMMUNITY_AREA_NUMBER from chicago_crime_data 
#group by COMMUNITY AREA NUMBER order by count(*) desc nulls last limit 1"""
#curr.execute(QUERY)
#crime_prone = curr.fetchall()
#crime_prone_pd = pd.DataFrame()
#for row in range(len(crime_prone)):
#    comm_num = str(crime_prone[row][0])
#    prone = crime_prone[row][1]
#    crime_prone_pd = crime_prone_pd.append({'COMMUNITY AREA NUMBER':comm_num,'CRIME PRONE':prone},ignore_index=True)
#max_comm_num = crime_prone_pd['CRIME PRONE'].max()
#max_comm_num

### Problem 9
##### Use a sub-query to find the name of the community area with highest hardship index.

In [12]:
curr.execute('select COMMUNITY_AREA_NAME,HARDSHIP_INDEX from census_data where HARDSHIP_INDEX = (select max(HARDSHIP_INDEX) from census_data)')
high_comm = curr.fetchall()
print('{} is the community area with the highest hardship index with index {}'.format(high_comm[0][0],high_comm[0][1]))

Riverdale is the community area with the highest hardship index with index 98


### Problem 10
##### Use a sub-query to determine the Community Area Name with most number of crimes?

In [13]:
query = """SELECT COMMUNITY_AREA_NAME, COUNT(*) AS total_records
FROM census_data
WHERE COMMUNITY_AREA_NUMBER = (
    SELECT COMMUNITY_AREA_NUMBER
    FROM chicago_crime_data
    GROUP BY COMMUNITY_AREA_NUMBER
    ORDER BY COUNT(*) DESC
    LIMIT 1
)
GROUP BY COMMUNITY_AREA_NAME"""
curr.execute(query)
print('{} is the community area with the most number of crime'.format(curr.fetchall()[0][0]))

Austin is the community area with the most number of crime


In [14]:
curr.close()
mydb.close()